# Impacts of Optimisation on Asymptotic Convergence of Graph Neural Networks

In this Google Colab, we explore the effects of optimisation procedures on the asymptotic convergence of GNNs, as the size of the input graph increases. It is shown in [Adam-Day, 2024](https://arxiv.org/abs/2403.03880), that Graph Neural Networks, when viewed as probabilistic classifiers (i.e. having softmax as the last layer, to give the probability that an input belongs to a class), converge to a fixed vector almost surely, as the size of the input graph tends to infinity. We now look at how this convergence changes under optimisation.

The full article is a mini-project for Graph Representation Learning.



# 1) Installing the Relevant Libraries

Below we install the relevant libraries and methods, such as PyTorch Geometric, NetworkX, NumPy and MatPlotLib.

In [1]:
%%capture
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import GPSConv
from torch_geometric.data import Data
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_add_pool
from scipy.stats import binom
import networkx as nx
from torch_geometric.utils import from_networkx
from torch_geometric.data import DataLoader
import matplotlib.pyplot as plt

# 2) Graph Neural Network Models

In this section we provide the classes for the GNNs we use. We implement the GCN and GAT architectures. In the paper we do not use GCN, yet we keep it here.

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        # self.conv3 = GCNConv(hidden_dim, hidden_dim)
        # self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch):
        # First GCN layer with ReLU activation
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Second GCN layer with ReLU activation
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        # # Third GCN layer with ReLU activation
        # x = self.conv3(x, edge_index)
        # x = F.relu(x)
        # Global mean pooling and fully connected layer
        x = global_mean_pool(x, batch)  # Aggregate node features
        # x = self.fc(x)
        return F.softmax(x, dim=-1)

In [4]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=4):
        super(GAT, self).__init__()

        # First GAT layer
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0)
        # Second GAT layer
        self.gat2 = GATConv(hidden_dim * heads, output_dim, heads=1, dropout=0)
        # Fully-connected layer
        # self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch):
        # Apply first GAT layer with ReLU activation
        x = F.relu(self.gat1(x, edge_index))
        # Apply second GAT layer
        x = self.gat2(x, edge_index)
        # x = F.relu(x)

        # Global mean pooling
        x = global_mean_pool(x, batch)  # Aggregate node features
        # x = self.fc(x)
        return F.softmax(x, dim=-1)

# 3) Generating Erdős–Rényi Graphs

In this section, we implement various classes of Erdős–Rényi graphs, and their conversion to a PyTorch Geometric data type.

There are various classification modes we analyse, and it can be picked from the following:
- "parity" - classifies as 1 if it has an even number of nodes, 0 if odd
- "average_degree" - classifies as 1 if it has more than the expected number of degrees, 0 otherwise

There are also two modes of generating, one where you can specify the probability, and one with inverse probability where the probability is automatically assigned as $\frac{1}{n}$. The latter provides better training in the average degree classification.

In [5]:
def generate_erdos_renyi_graph(num_nodes, probability, feature_dim, mode="None"):
    # Generate a random Erdos-Renyi graph using NetworkX
    G = nx.erdos_renyi_graph(num_nodes, probability)
    # Add random node features
    node_features = torch.rand((num_nodes, feature_dim))
    # Convert to PyTorch Geometric format
    data = from_networkx(G)
    data.x = node_features
    data.num_nodes = num_nodes

    if mode == "None":
      raise ValueError("Mode must be specified")

    if mode == "parity":
      if num_nodes % 2 == 0:
        data.y = torch.tensor([1])
      else:
        data.y = torch.tensor([0])

    if mode == "average_degree":
      average_degree = torch.tensor([sum(dict(G.degree).values()) / num_nodes])
      data.avg_deg = average_degree
      if average_degree >= (num_nodes - 1) * probability:
        data.y = torch.tensor([1])
      else:
        data.y = torch.tensor([0])

    return data

In [6]:
def generate_erdos_renyi_inverse_prob(num_nodes,  feature_dim, mode="None"):
  return generate_erdos_renyi_graph(num_nodes, 1/num_nodes, feature_dim, mode=mode)

# 4) Utils

This section provides the utils for counting the class memberships, model training, evaluation and plotting.

In [7]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

In [8]:
def evaluate(model, dataloader, device):
    model.eval()
    size = len(dataloader.dataset)
    correct = 0
    with torch.no_grad():
        for data in dataloader:
            data.to(device)
            pred = model(data.x, data.edge_index, data.batch)
            correct += (pred.argmax(-1) == data.y).count_nonzero()

    return correct / len(dataloader.dataset)

In [9]:
def class_frequency(dataloader):

  freq = {}

  for batch in dataloader:
    batch_classes = batch.y
    for y in batch_classes:
      y = y.item()
      if y not in freq:
        freq[y] = 1
      else:
        freq[y] += 1

  return freq

In [67]:
def get_softmax_outputs(model, plot_settings, device):

    model.eval()
    outputs = []

    all_graph_sizes = plot_settings['all_graph_sizes']
    uncomputed_graph_sizes = plot_settings['uncomputed_graph_sizes']
    pre_computed_graph_sizes = plot_settings['pre_computed_graph_sizes']
    sample_per_graph = plot_settings['sample_per_graph']
    feature_dim = plot_settings['feature_dim']
    edge_prob = plot_settings['edge_prob']
    mode = plot_settings['mode']
    inverse_prob = plot_settings['inverse_prob']

    all_outputs = []

    with torch.no_grad():
        for size in uncomputed_graph_sizes:

          size_outputs = []

          for _ in range(sample_per_graph):
            if inverse_prob:
                data = generate_erdos_renyi_inverse_prob(num_nodes=size, feature_dim=feature_dim, mode=mode).to(device)
            else:
                data = generate_erdos_renyi_graph(num_nodes=size, probability=edge_prob, feature_dim=feature_dim, mode=mode).to(device)
            out = model(data.x, data.edge_index, data.batch)
            size_outputs.append(out.cpu().numpy()[0])
            # Out gives a list of lists (in case the data is a batch). Hence use out.numpy()[0] to get the first (and only) element.

          all_outputs.append(size_outputs)
          # averages along the 0th axis (i.e. row average)
          average_output = np.average(size_outputs, 0)
          outputs.append(average_output)
          print(f'Size {size} sample computed.')

        for size in pre_computed_graph_sizes:

          file_name = 'ER_graphs_10_dim_' + str(size) + '_nodes.pkl'
          graphs = load_pickle(file_name)

          size_outputs = []

          for data in graphs:
            data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            size_outputs.append(out.cpu().numpy()[0])
            # Out gives a list of lists (in case the data is a batch). Hence use out.numpy()[0] to get the first (and only) element.


          all_outputs.append(size_outputs)
          # averages along the 0th axis (i.e. row average)
          average_output = np.average(size_outputs, 0)
          outputs.append(average_output)
          print(f'Size {size} sample computed.')

    outputs = np.asarray(outputs)
    return all_outputs, outputs


def plot_probabilities(all_graph_sizes, outputs):

    plt.figure(figsize=(10, 6))

    for i in range(outputs.shape[1]):
      column = outputs[:, i]
      plt.plot(all_graph_sizes, column, label=f'Class {i}')

    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Class Probabilities", fontsize=12)
    ax = plt.gca()
    # ax.set_xscale('log')
    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1, 0.1))
    plt.title("Asymptotic Class Probabilities", fontsize=14)
    plt.grid(True)
    plt.legend(fontsize=10)
    plt.show()

In [11]:
def dataloader_generator(size_range, number_of_graphs_per_size, feature_dim, split_prob = None, mode="None", prob = None, inverse_prob = False, batch_size=32, shuffle=True):
  data = []
  for i in range(size_range[0], size_range[1] + 1):
    for _ in range(number_of_graphs_per_size):
      if inverse_prob:
        graph = generate_erdos_renyi_inverse_prob(i, feature_dim, mode)
        if split_prob:
          binary_label_graphs(graph, split_prob, graph.num_nodes, 1 / graph.num_nodes)
        data.append(graph)
      if not inverse_prob:
        if prob is None:
          raise ValueError("Cannot have probability as None, if not using the inverse_prob setting!")
          graph = generate_erdos_renyi_inverse_prob(i, prob, feature_dim, mode)
          if split_prob:
            binary_label_graphs(graph, split_prob, graph.num_nodes, prob)
        data.append(graph)

  data_loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle)

  return data, data_loader

In [12]:
def binomial_cdf_split(split_prob, n, p):

  split = 0

  while binom.cdf(split + 1, n, p) < split_prob[0]:
    split += 1

  if split_prob[0] - binom.cdf(split, n, p) <= binom.cdf(split + 1, n, p) - split_prob[0]:
    split1 = split
  else:
    split1 = split + 1

  while binom.cdf(split + 1, n, p) - binom.cdf(split1, n, p) < split_prob[1]:
    split += 1

  if split_prob[1] - binom.cdf(split, n, p) <= binom.cdf(split + 1, n, p) - split_prob[1]:
    split2 = split
  else:
    split2 = split + 1

  return [split1, split2]

# s1, s2 = binomial_cdf_split([0.6, 0.3], 10*9, 0.1)
# print(binom.cdf(s1, 10*9, 0.1))
# print(binom.cdf(s2, 10*9, 0.1))

# 5) Pickling Erdős–Rényi Graphs For Fast Computation

In this section, we pickle and load the Erdős–Rényi Graphs to compute it quickly during the experiments.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pickle

# Save Pickle to the 'gnn_convergence' subfolder
def save_pickle(data, file_name):

  # Define the file path
  file_path = f'./drive/MyDrive/gnn_convergence/{file_name}'

  # Save data to the file
  with open(file_path, 'wb') as f:
      pickle.dump(data, f)

  print(f"File saved to {file_path}")

# Load Pickle from 'gnn_convergence' subfolder
def load_pickle(file_name):

  # Define the file path
  file_path = f'./drive/MyDrive/gnn_convergence/{file_name}'

  with open(file_path, 'rb') as f:
    loaded_data = pickle.load(f)

  return loaded_data

def load_graphs_of_size(size):
  file_name = 'ER_graphs_10_dim_' + str(size) + '_nodes.pkl'
  return load_pickle(file_name)


In [15]:
# Here, we generate and pickle 20 graphs for each graph size in the ER(n, 1/n) model
# The features here are all randomly generated, 10 dimensional features

all_graph_sizes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 700, 1000, 1200, 1500, 2000, 2200, 2500, 2700, 3000, 3300, 3500, 3700, 4000, 4200, 4400, 4600, 4800, 5000,
                   5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 11000, 12000, 13000, 15000, 17000, 20000, 22000, 25000, 27000, 30000, 32000,
                   35000, 37000, 40000, 42000, 44000, 46000, 48000, 50000]
uncomputed_graph_sizes = [10, 20, 30, 40, 50, 60, 70, 80, 90]
precomputed_graph_sizes = [100, 200, 300, 400, 500, 700, 1000, 1200, 1500, 2000, 2200, 2500, 2700, 3000, 3300, 3500, 3700, 4000, 4200, 4400, 4600, 4800, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 11000, 12000, 13000, 15000, 17000, 20000, 22000, 25000,
                              27000, 30000, 32000, 35000, 37000, 40000, 42000, 44000, 46000, 48000, 50000]

graph_per_size = 20
feature_dim_samples = 10

plot_settings = {
    'all_graph_sizes' : all_graph_sizes,
    'uncomputed_graph_sizes' : uncomputed_graph_sizes,
    'sample_per_graph' : 20,
    'pre_computed_graph_sizes' : precomputed_graph_sizes,
    'edge_prob' : 0,
    'feature_dim' : 10,
    'mode' : 'average_degree',
    'inverse_prob' : True
}

In [16]:
# Code to precompute Erdos-Renyi graphs to save time

# for size in uncomputed_graph_sizes:
#   size_graphs = []
#   for _ in range(graph_per_size):
#     graph = generate_erdos_renyi_inverse_prob(size, feature_dim_samples, mode='average_degree')
#     size_graphs.append(graph)
#
#   file_name = 'ER_graphs_10_dim_' + str(size) + '_nodes.pkl'
#   save_pickle(size_graphs, file_name)
#   print(f'Graphs of size {size} is now saved in Google Drive.')

In [17]:
def binary_label_graphs(graph, split_prob, number_of_nodes, prob):

  sp1, sp2 = binomial_cdf_split(split_prob, number_of_nodes * (number_of_nodes - 1), prob)
  avg_split_1 = sp1 / number_of_nodes
  avg_split_2 = sp2 / number_of_nodes
  if graph.avg_deg <= avg_split_1:
    graph.y = 0
  elif avg_split_1 < graph.avg_deg <= avg_split_2:
    graph.y = 1
  else:
    graph.y = 2

In [18]:
def dataloader_binary_label_inverse_prob(dataloader, split_prob):

  for batch in dataloader:
    for graph in batch:
      graph_size = graph.num_nodes
      binary_label_graphs(graph, split_prob, graph_size, 1 / graph_size)

# 6) Experiments Setup

This is the main experiment setup. In this section, we consider $ER(n, \frac{1}{n})$ graphs and train a GNN to classify whether the average degree falls into certain buckets. Hyperparameters can be changed here.

In [19]:
# Initialising all parameters for the training

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model & dataset initialisation
edge_prob = 0.1
split_prob = [0.55, 0.28]
feature_dim = 10
mode = "average_degree"
inverse_prob = True

# training and test settings
train_number_of_graphs_per_size = 50
train_range = [10, 100]
in_bound_test_number_of_graphs_per_size = 10
in_bound_test_range = [10, 100]
out_bound_test_number_of_graphs_per_size = 10
out_bound_test_range = [500, 600]
batch_size = 32
lr = 0.001

In [ ]:
# Create synthetic datasets
train_data, train_loader = dataloader_generator(size_range=train_range, number_of_graphs_per_size=train_number_of_graphs_per_size, feature_dim=feature_dim, split_prob=split_prob,
                                    mode=mode, prob=edge_prob, inverse_prob=inverse_prob, batch_size=batch_size, shuffle=True)

test_data, in_bound_test_loader = dataloader_generator(size_range=in_bound_test_range, number_of_graphs_per_size=in_bound_test_number_of_graphs_per_size, feature_dim=feature_dim, split_prob=split_prob,
                                            mode=mode, prob=edge_prob, inverse_prob=inverse_prob, batch_size=batch_size, shuffle=False)

# One can also create out of bound test set as below

# out_bound_test_loader = dataloader_generator(size_range=out_bound_test_range, number_of_graphs_per_size=out_bound_test_number_of_graphs_per_size, feature_dim=feature_dim,
#                                             mode=mode, prob=edge_prob, inverse_prob=inverse_prob, batch_size=batch_size, shuffle=False)

# ==================
# If the train data is already in the computer, can just unpickle it

# train_data = load_pickle('train_data_53_20_27.pkl')
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
#
# test_data = load_pickle('test_data.pkl')
# in_bound_test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# ==================

# Checks the frequency and proportions of each class
print("SANITY CHECK")
freq = class_frequency(train_loader)
print(freq)
print(f'Class 0 Ratio: {freq[0] / (freq[0] + freq[1] + freq[2])}')
print(f'Class 1 Ratio: {freq[1] / (freq[0] + freq[1] + freq[2])}')
print(f'Class 2 Ratio: {freq[2] / (freq[0] + freq[1] + freq[2])}')

In [21]:
def model_train_loop(model, epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=lr)

  for epoch in range(epochs):
    loss = train_epoch(model, train_loader, criterion, optimizer, device)
    print(f'Train Accuracy: {evaluate(model, train_loader, device)}')
    print(f'In-Bound Test Accuracy: {evaluate(model, in_bound_test_loader, device)}')
    # print(f'Out-Bound Test Accuracy: {evaluate(model, out_bound_test_loader, device)}')
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
    print("=============")

# 7) Varying Epoch Training

This is a sample code for training 5 different models with 10 eppchs of training. This code can be reused to make other models. Here we also pickle the data, to be used in plotting later.

In [ ]:
model_count = 5
models = [GAT(input_dim=feature_dim, hidden_dim=64, output_dim=3).to(device) for i in range(model_count)]

for model in models:
  model_train_loop(model, epochs=10)

all_outputs_10_epoch, softmax_outputs_10_epochs = [], []

for model in models:
  all_outputs, outputs = get_softmax_outputs(model, plot_settings, device)
  all_outputs_10_epoch.append(all_outputs)
  softmax_outputs_10_epochs.append(outputs)

save_pickle(softmax_outputs_19_epochs, 'softmax_outputs_10_epochs.pkl')
save_pickle(all_outputs_10_epoch, 'all_outputs_10_epochs.pkl')

# 8) Plotting

Below is the code for loading the data, and plotting the class probabilities and standard deviations.

In [ ]:
softmax_outputs_10_epochs = load_pickle('softmax_outputs_10_epochs.pkl')
softmax_outputs_20_epochs = load_pickle('softmax_outputs_20_epochs.pkl')
softmax_outputs_50_epochs = load_pickle('softmax_outputs_50_epochs.pkl')
softmax_outputs_100_epochs = load_pickle('softmax_outputs_100_epochs.pkl')
softmax_outputs_200_epochs = load_pickle('softmax_outputs_200_epochs.pkl')

all_outputs_10_epoch = load_pickle('all_outputs_10_epochs.pkl')
all_outputs_20_epoch = load_pickle('all_outputs_20_epochs.pkl')
all_outputs_50_epoch = load_pickle('all_outputs_50_epochs.pkl')
all_outputs_100_epoch = load_pickle('all_outputs_100_epochs.pkl')

In [ ]:
# CLASS PROBABILITIES

def model_avg(softmax_out):
  model_1_output = np.asarray(softmax_out[0])
  model_2_output = np.asarray(softmax_out[1])
  model_3_output = np.asarray(softmax_out[2])
  model_4_output = np.asarray(softmax_out[3])
  model_5_output = np.asarray(softmax_out[4])

  return (model_1_output + model_2_output + model_3_output + model_4_output + model_5_output) / 5

avg_mod_out_0 = model_avg(softmax_outputs_NO_epochs)[:,0]
avg_mod_out_10 = model_avg(softmax_outputs_10_epochs)[:,0]
avg_mod_out_20 = model_avg(softmax_outputs_20_epochs)[:,0]
avg_mod_out_50 = model_avg(softmax_outputs_50_epochs)[:,0]
avg_mod_out_100 = model_avg(softmax_outputs_100_epochs)[:,0]

x_plot = all_graph_sizes

plt.plot(x_plot, avg_mod_out_0, label='No Training', linestyle='-', color='orange')
plt.plot(x_plot, avg_mod_out_10, label='10 Epochs', linestyle='-', color='blue')
plt.plot(x_plot, avg_mod_out_20, label='20 Epochs', linestyle='-', color='red')
plt.plot(x_plot, avg_mod_out_50, label='50 Epochs', linestyle='-', color='green')
plt.plot(x_plot, avg_mod_out_100, label='100 Epochs', linestyle='-', color='purple')

plt.xlabel('Graph Size')
plt.ylabel('Class Probability')
plt.yticks(np.arange(0.25, 1, 0.1))
plt.title('Top Class Probability with Different Training')
plt.legend()

# Display the plot
plt.grid(True)
plt.show()


In [ ]:
# CLASS PROBABILITIES

model_1_output = np.asarray(softmax_outputs_100_epochs[0])
model_2_output = np.asarray(softmax_outputs_100_epochs[1])
model_3_output = np.asarray(softmax_outputs_100_epochs[2])
model_4_output = np.asarray(softmax_outputs_100_epochs[3])
model_5_output = np.asarray(softmax_outputs_100_epochs[4])

avg_mod_out = (model_1_output + model_2_output + model_3_output + model_4_output + model_5_output) / 5

y_class_1 = avg_mod_out[:,0]
y_class_2 = avg_mod_out[:,1]
y_class_3 = avg_mod_out[:,2]

x_plot = all_graph_sizes

plt.plot(x_plot, y_class_1, label='Class 0', linestyle='-', color='green')
plt.plot(x_plot, y_class_2, label='Class 1', linestyle='-', color='blue')
plt.plot(x_plot, y_class_3, label='Class 2', linestyle='-', color='red')

plt.xlabel('Graph Size')
plt.ylabel('Class Probabilities')
plt.title('Class Probabilities For Increasing Graph Size')
plt.legend()

plt.grid(True)
plt.show()

In [ ]:
# STANDARD DEVIATION

def process_model(model_outs):

  col_1_means = []
  col_2_means = []
  col_3_means = []

  for samples_20 in model_outs:
    arr = np.asarray(samples_20)
    col_1_means.append(np.mean(arr[:,0]))
    col_2_means.append(np.mean(arr[:,1]))
    col_3_means.append(np.mean(arr[:,2]))

  mean_col_1 = np.mean(col_1_means)
  mean_col_2 = np.mean(col_2_means)
  mean_col_3 = np.mean(col_3_means)

  col_1_sds = []
  col_2_sds = []
  col_3_sds = []

  for samples_20 in model_outs:
    arr = np.asarray(samples_20)
    std_col_1 = col_1_sds.append(np.std(np.abs(arr[:,0] - mean_col_1)))
    std_col_2 = col_2_sds.append(np.std(np.abs(arr[:,1] - mean_col_2)))
    std_col_3 = col_3_sds.append(np.std(np.abs(arr[:,2] - mean_col_3)))

  return np.array(col_1_sds) + np.array(col_2_sds) + np.array(col_3_sds)

def avg_stdev(multiple_model_comp):
  stdevs = process_model(multiple_model_comp[0])
  for model_outs in multiple_model_comp[1:]:
    stdevs += process_model(model_outs)
  return stdevs / len(multiple_model_comp)

In [ ]:
# the y_NO_epoch is not pickled - one needs to generate it
y_NO_epoch = avg_stdev(all_outputs_NO_epoch)
y_10_epoch = avg_stdev(all_outputs_10_epoch)
y_20_epoch = avg_stdev(all_outputs_20_epoch)
y_50_epoch = avg_stdev(all_outputs_50_epoch)
y_100_epoch = avg_stdev(all_outputs_100_epoch)

x_plot = all_graph_sizes

plt.plot(x_plot, y_NO_epoch, label='No Training', linestyle='-', color='orange')
plt.plot(x_plot, y_10_epoch, label='10 Epochs', linestyle='-', color='blue')  # Add label and style
plt.plot(x_plot, y_20_epoch, label='20 Epochs', linestyle='-', color='red')
plt.plot(x_plot, y_50_epoch, label='50 Epochs', linestyle='-', color='green')
plt.plot(x_plot, y_100_epoch, label='100 Epochs', linestyle='-', color='purple')

plt.xlabel('Graph Size')
plt.ylabel('Standard Deviation')
plt.xscale('log')
plt.title('Standard Deviation From Class Mean')
plt.legend()

plt.grid(True)
plt.show()